# Jigsaw Unintended Bias in Toxicity Classification

Team members: Tanmay Sardesai, Prateek Malhotra

## Introduction

The Conversation AI team founded by Jigsaw and Google builds technology to protect voices in conversations. Their main machine learning focus is to identify toxicity in online conversations. Toxicity is defined as anything rude, disrespectful or something likely to make someone leave a discussion. Last year Jigsaw hosted a toxicity classification challenge where participants built models to recognize toxicity along with several subtypes. Some of the top scoring models were very baised. This year's competition is to detect toxicity in comments while building models that operate fairly across a range of conversations.  

Here is an excerpt from the competition page on kaggle.com that perfectly explains the problem and goal.

> When the Conversation AI team first built toxicity models, they found that the models incorrectly learned to associate the names of frequently attacked identities with toxicity. Models predicted a high likelihood of toxicity for comments containing those identities (e.g. "gay"), even when those comments were not actually toxic (such as "I am a gay woman"). This happens because training data was pulled from available sources where unfortunately, certain identities are overwhelmingly referred to in offensive ways. Training a model from data with these imbalances risks simply mirroring those biases back to users.
> 
> In this competition, you're challenged to build a model that recognizes toxicity and minimizes this type of unintended bias with respect to mentions of identities. You'll be using a dataset labeled for identity mentions and optimizing a metric designed to measure unintended bias. Develop strategies to reduce unintended bias in machine learning models, and you'll help the Conversation AI team, and the entire industry, build models that work well for a wide range of conversations.

Disclaimer: The dataset for this competition contains text that may be considered profane, vulgar, or offensive which is why some examples in this report may contain such words

## Evaluation Metric

This competition evaluates the our submission based on a new metric described in Borkman et al(2019). The evaluation metric is based on 2 values.

#### 1. Overall AUC
This is the ROC-AUC for the full evaluation set.

#### 2. Generalized Mean of Bias AUCs

To understand Generalized Mean of Bias AUCs we first have to discuss the following 3 types of Bias AUCs
1. Subgroup AUC:
    Here we restrict the dataset to be all the comments mentioning a specific identity. A low score here means our model cannot differentiate between toxic and non-toxic comments for this identity
2. BPSN (Background Positive, Subgroup Negative) AUC:
    Here we restrict the dataset to be all the non-toxic comments mentioning the identity and all the toxic comments not mentioning the identity. A low score here means that the model predicts high toxicity than it should for non-toxic examples mentioning the identity
3. BNSP (Background Negative, Subgroup Positive) AUC:
    Here we do opposite of BPSN. We restrict the dataset to be all the toxic comments mentioning the identity and all the non-toxic comments not mentioning the identity. A low score here means that the model predicts low toxicity than it should for non-toxic examples mentioning the identity.

Generalized Mean of Bias AUCs is defined as follows

$$ M_p (m_s) = \left(\frac{1}{N} \sum_{s=1}^{N} m_s^p \right)^{\frac{1}{p}} $$

where: <br>
$ M_p = $ the pth power-mean function <br>
$ m_s = $ the bias metric m calulated for subgroup s <br>
$ N = $ number of identity subgroups <br>

For this competition the organizers used a p value of -5 to encourage competitors to improve the model for the identity subgroups with the lowest model performance.

### Final Metric

The final metric is weighted average of the 4 values above: Overall AUC, 3 Bias AUCs

$$ score = w_0 * AUC_{overall} + \sum_{a=1}^A w_a M_p (m_{s,a}) $$

where: <br>
$ A = $ number of sub metrics = 3 <br>
$ m_{s,a} = $ bias metric for identity subgroup **s** using submetric **a** <br>
$ w_a = $ a weighting for the relative importance of each submetric; all four **w** values set to 0.25 in this competition. <br>

## Dataset

The training dataset consists of 1.8 million text comments from the [Civil Comments](https://medium.com/@aja_15265/saying-goodbye-to-civil-comments-41859d3a2b1d) platform. Each comment has a toxicity label that models have to predict. The labels are values between 0 and 1 as they are average of toxicity value given by multiple human annotators. For evaluation target >= 0.5 is considered as a toxic comment.

The dataset also provides additional information that define the type of toxicity. The labels here are 

* severe_toxicity
* obscene
* threat
* insult
* identity_attack
* sexual_explicit

These labels are not required to be predicted but we noticed that if these labels are also used during training our model performs better. The model also provide identity values for multiple identities. Only the ones with at least 500 examples are used. The number of identities is 9 and they are as follows:

* male 
* female
* homosexual_gay_or_lesbian
* christian
* jewish
* muslim
* black
* white
* psychiatric_or_mental_illness

In the next 2 section we will describe our models. As we want to keep the length of this report at a minimum we are not going to talk about preprocessing and building of embedding matrix. For all the models that we trained, we built the embedding matrix by concatenating Glove and Crawl 300D vectors. Preprocessing was done to make sure that the vocab in the comments matches to the vocab that glove and crawl were trained on so that we have less number of unknown words.

## Bi-LSTM based Model

Our first model uses Bi-LSTM Layers. We started building on top of a baseline model publically shared by one of the participants in the competition. https://www.kaggle.com/kunwar31/simple-lstm-with-identity-parameters-fastai

Our contributions to the above model comes by applying the techniques described in Beutel et al(2017). We use the information that we have about identities and train the model such that it doesn't base its decisions on those identities. This model uses idea of adversarial training. Let X be our train input dataset. Let S be a subset of X for which we have observed identities. Also let Y be the toxicity values that we are trying to predict and Z be the values for observed identities.

The findings in Beutel et al. can be best described by looking at the following image.

<img src="https://i.ibb.co/hggQqRZ/Screen-Shot-2019-06-09-at-10-51-24-PM.png" alt="Beutel Model" style="width: 500px;"/>

We train the model with the following objective 

$$ min \left[ \sum_{x,y \in X} L_y(f(g(x)), y) + L_z(a(J_{\lambda}(g(x))) ,z)  \right] $$

Here g(x) is the shared model with the embeddings and hidden layers. In our case it is made of 2 LSTM layers, mean pooling, max pooling and 2 linear layers as defined below in NeuralNet class. Then we have f() and a() which are Linear layers that predict y and z respectively. In our model a() is just a single liner layer but f() is two linear layers, 1 for predicting the target toxic value and other for predicted auxilary target values. Finally we have $J_{\lambda}$ which is similar to identity function in forward pass but returns negative gradient in the backward pass as defined in GradReverse class and grad_reverse function.

In [7]:
"""Import statements so that the code below doesn't give errors in the notebook"""

import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
"""
We use the following code to apply the negative gradient during backward pass while training.
"""
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg()

def grad_reverse(x, lambdax = 0.1):
    return lambdax * GradReverse.apply(x)

In [9]:
"""
LSTM Based model
"""
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x
    
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix, num_aux_targets, num_identity_targets):
        super(NeuralNet, self).__init__()
        embed_size = embedding_matrix.shape[1]
        
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)
        self.linear_id_out1 = nn.Linear(DENSE_HIDDEN_UNITS, num_identity_targets)
        
    def forward(self, x, lengths=None):
        h_embedding = self.embedding(x.long())
        h_embedding = self.embedding_dropout(h_embedding)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        # global average pooling
        avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)
        
        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_conc))
        h_conc_linear2  = F.relu(self.linear2(h_conc))
        
        hidden = h_conc + h_conc_linear1 + h_conc_linear2
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        
        hidden = grad_reverse(hidden, lambdax = 0.1)
        id_result = self.linear_id_out1(hidden)
        out = torch.cat([result, aux_result, id_result], 1)
        
        return out

The above changes gave us an improvement over the base LSTM model by increasing the score from 0.93524 to 0.93771

## BERT based Model

As the above concepts can be generalized for any type of model. We modified the Vanilla BERT classification Kernel [5] such that it is treated as g(x) and reused the linear layers model for f(x) and a(x). This improved the score of 0.92987 from Vanilla BERT classification Kernel to 0.93827. Due to computational constraints our BERT model was only trained on 1.2M comments out of the 1.8M

In [10]:
class BERT_mod(nn.Module):
    def __init__(self):
        super(BERT_mod, self).__init__()
        self.BERT = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels=512)
        self.linear_id = nn.Linear(512, 9)
        self.linear_out = nn.Linear(512, 1)
        self.linear_aux_out = nn.Linear(512, 6)
        
    def forward(self, input, attention_mask, labels):
        hidden = self.BERT(input, attention_mask=attention_mask, labels=labels)
        
        hidden = F.relu(hidden)
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        
        hidden = grad_reverse(hidden, lambdax = 1.0)
        id_preds = self.linear_id(hidden)
        
        return torch.cat([result, aux_result, id_preds], 1)

## Things we tried that didn't give good results

Here is a list of things we tried which didn't improve the model

* Data Augmentation <br>
    We tried replacing words like gay, lesbian with words like straight and heterosexual as part of preprocessing. This didn't give us any improvement. One of the possible reasons could be that there were other context words in the comment that we were replacing. For example a comment like "I hate women. She is very bad" becomes "I hate men. She is very bad". Replacing all the context words in a dataset of size 1.8M is unrealistic.
    
* Input statistical data about comment along with the word vectors. <br>
    Here we tried to pass a count of number of profanities used from this list at https://github.com/RobertJGabriel/Google-profanity-words. Along with this we also passed information like number of capital words, number of typos/unkown words among other things. This helped increase the BPSN score, i.e our model was able to predict appropriate toxicity value of non-toxic example for all identities but it decreased the BNSP score, i.e our model was not able to predict appropriate toxicity value of toxic examples for all identities.
    
* Masking <br>
    We tried masking 15% of the "seed" words, words related to protected identity generated by using word embedding vector. This also didn't give us any improvement. We couldn't not figure out the reason behind this but suspect it has the same issues as we saw in data augmentation technique

# Conclusion

Currently we are ranked at 581/2695 (top 22%) with a score of 0.93827 by using our fine tuned BERT model which we trained on 1.26M out of the 1.8M comments due to computational and time constraints. There are still 17 days left before the competition ends. In the next two weeks we will fine tune our BERT model on all the training data along with some hyper parameter tuning. Our final submision will be an ensemble of some LSTM and some BERT models with different hyper parameters and random seed values. All the code will be available on github at https://github.com/tnsardesai/DebiasingToxicityClassification

# References

[1] Daniel Borkan, Lucas Dixon, Jeffrey Sorensen, Nithum Thain, Lucy Vasserman. 2019. Nuanced Metrics for Measuring Unintended Bias with Real Data for Text Classification. https://arxiv.org/pdf/1903.04561.pdf

[2] https://www.kaggle.com/kunwar31/simple-lstm-with-identity-parameters-fastai

[3] https://github.com/RobertJGabriel/Google-profanity-words

[4] https://www.kaggle.com/dborkan/benchmark-kernel

[5] https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila

[6] https://www.kaggle.com/christofhenkel/how-to-preprocessing-for-glove-part2-usage